## Imports / Libraries

In [ ]:
from bs4 import BeautifulSoup 
from time import sleep 
import pandas as pd 
from selenium import webdriver 
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Macro definition - Webscraping

In [ ]:
def get_page(page):
    url = f"https://www.iaaf.org/world-rankings/overall-ranking/men?regionType=world&page={page}&rankDate=2019-08-27&limitByCountry=0"
    option = webdriver.ChromeOptions().add_argument('--incognito')
    browser = webdriver.Chrome(executable_path='./chromedriver', options=option)  
    browser.get(url)
    sleep(5)
    page_source = browser.page_source
    soup = BeautifulSoup(page_source, 'html5lib')
    browser.close()
    return soup


## Loop - Macro execution

In [ ]:
iaaf_database = pd.DataFrame()

for i in range(121):
    temp = get_page(i+1)
    
    interesting_tds = ['Rank', 'Competitor', 'DOB', 'Nat', 'score', 'EventList']
    Rank = list()
    Competitor = list()
    DOB = list()
    Nat = list()
    Score = list()
    Eventlist = list()

    for td in temp.find_all("td"):
        if td.get('data-th') == 'Rank':
            Rank.append(td.text.strip())

    for td in temp.find_all("td"):
        if td.get('data-th') == 'Competitor':
            Competitor.append(td.text.strip())
        
    for td in temp.find_all("td"):
        if td.get('data-th') == 'DOB':
            DOB.append(td.text.strip())
        
    for td in temp.find_all("td"):
        if td.get('data-th') == 'Nat':
            Nat.append(td.text.strip())
        
    for td in temp.find_all("td"):
        if td.get('data-th') == 'score':
            Score.append(td.text.strip())
        
    for td in temp.find_all("td"):
        if td.get('data-th') == 'EventList':
            Eventlist.append(td.text.strip())
            
    data_athletes = {'Rank':Rank,'Competitor':Competitor,'DOB':DOB,'Nat':Nat,'Score':Score,'Eventlist':Eventlist}
    data_athletes_df = pd.DataFrame(data=data_athletes)
    iaaf_database = iaaf_database.append(data_athletes_df)
    
    # Printing dataframe length and the page # just done, for checking
    print(len(iaaf_database))
    print("page", i+1, "done")

#Storing the monthly data into a csv
iaaf_database["DateRank"] = datetime.date(2019, 8, 27)
iaaf_database.to_csv(r'C:\Users\Victor Leo\Desktop\WeCloud - Python for Data Science\Project 1\iaaf_database_20190827.csv')

## Compiling monthly scores into one dataset

In [ ]:
january = pd.read_csv("C:/Users/Victor Leo/Desktop/WeCloud - Python for Data Science/Project 1/iaaf_database_20190129.csv", parse_dates=['DOB','DateRank'])
february = pd.read_csv("C:/Users/Victor Leo/Desktop/WeCloud - Python for Data Science/Project 1/iaaf_database_20190226.csv", parse_dates=['DOB','DateRank'])
march = pd.read_csv("C:/Users/Victor Leo/Desktop/WeCloud - Python for Data Science/Project 1/iaaf_database_20190326.csv", parse_dates=['DOB','DateRank'])
april = pd.read_csv("C:/Users/Victor Leo/Desktop/WeCloud - Python for Data Science/Project 1/iaaf_database_20190430.csv", parse_dates=['DOB','DateRank'])
may = pd.read_csv("C:/Users/Victor Leo/Desktop/WeCloud - Python for Data Science/Project 1/iaaf_database_20190528.csv", parse_dates=['DOB','DateRank'])
june = pd.read_csv("C:/Users/Victor Leo/Desktop/WeCloud - Python for Data Science/Project 1/iaaf_database_20190625.csv", parse_dates=['DOB','DateRank'])
july = pd.read_csv("C:/Users/Victor Leo/Desktop/WeCloud - Python for Data Science/Project 1/iaaf_database_20190730.csv", parse_dates=['DOB','DateRank'])
august = pd.read_csv("C:/Users/Victor Leo/Desktop/WeCloud - Python for Data Science/Project 1/iaaf_database_20190827.csv", parse_dates=['DOB','DateRank'])

# Appending all datasets
complete = pd.concat([january,february,march,april,may,june,july,august], sort=False)
complete = complete[["Rank","Competitor","DOB","Nat","Score","Eventlist","DateRank"]]

# Calculating the age in years
complete["age"] = complete["DateRank"] - complete["DOB"]
complete["age"] = complete["age"]/np.timedelta64(1,'Y')

# Calculating the number of events
complete['Events']=complete['Eventlist'].str.split('[').str[0]
complete['Events']=complete['Events'].str.replace(",000", "000")
complete["number_events"] = complete["Events"].str.count(',') + 1


## Descriptive analysis - Countries

In [ ]:
# Descriptive tables
complete.describe(include="all")

# Scores histogram
complete["Score"].hist()

# Descriptives - Group by country - August/2019
complete2 = complete[complete["DateRank"]=="2019-08-27"]
complete2.groupby("Nat").aggregate({'Rank': 'count',
                                   'age': 'mean',
                                   'number_events': "mean",
                                   'Score': ('mean','std','sum')}).sort_values(by=("Score","sum"), ascending=False).head(10)

## Descriptive analysis - Athletes

In [ ]:
complete2.sort_values(by="Score", ascending=False)

# Selcting top 5 athletes
Kipchoge = complete[complete["Competitor"]=="Eliud KIPCHOGE"]
Lyles = complete[complete["Competitor"]=="Noah LYLES"]
Warholm = complete[complete["Competitor"]=="Karsten WARHOLM"]
Coleman = complete[complete["Competitor"]=="Christian COLEMAN"]
Walsh = complete[complete["Competitor"]=="Tomas WALSH"]

# Line plot - Top 5 athletes
plt.plot( "DateRank", "Score", data=Kipchoge, marker='o', markerfacecolor='blue', markersize=12, color='blue', linewidth=4, label="Kipchoge")
plt.plot( "DateRank", "Score", data=Lyles, marker='o', markerfacecolor='red', markersize=12, color='red', linewidth=4, label="Lyles")
plt.plot( "DateRank", "Score", data=Warholm, marker='o', markerfacecolor='skyblue', markersize=12, color='skyblue', linewidth=4, label="Warholm")
plt.plot( "DateRank", "Score", data=Coleman, marker='o', markerfacecolor='green', markersize=12, color='green', linewidth=4, label="Coleman")
plt.plot( "DateRank", "Score", data=Walsh, marker='o', markerfacecolor='brown', markersize=12, color='brown', linewidth=4, label="Walsh")
plt.legend()